In [23]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)


from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


Вводим токен, получаем список доступных аккаунтов

In [45]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'

token = TOKEN0


### Portfolio

Получаем портфель, список всех позиций на аккаунте

In [58]:
def get_accounts(token):
    with Client(token) as cl:
        accounts = cl.users.get_accounts()
    return accounts

def get_portfolio(token):
    accounts = get_accounts(token)
    acc_id = accounts.accounts[0].id
    with Client(token) as cl:
        port = cl.operations.get_portfolio(account_id=acc_id) 
    return port

def get_id_base(token):
    with Client(token) as cl:
        instruments = cl.instruments
        market_data = cl.market_data
    
        l = []
        for method in ['shares', 'currencies', 'futures', 'bonds', 'etfs']:
            for item in getattr(instruments, method)().instruments:
                l.append({
                    'ticker': item.ticker,
                    'figi': item.figi,
                    'type': method,
                    'name': item.name,
                    'cur' : item.currency,
                    'lot' : item.lot
                })
    
        df = pd.DataFrame(l)
    return df

def ticker_to_figi(ticker, df):
    dfx = df[df['ticker'] == ticker]   
    if dfx.shape[0] > 0 :
        figi = dfx['figi'].iloc[0]
        return figi
    else:
        return None

def figi_to_ticker(figi, df):
    dfx = df[df['figi'] == figi]   
 
    if dfx.shape[0] > 0 :
        ticker = dfx['ticker'].iloc[0]      
        return ticker
    else :
        return None

def figi_to_name(figi, df):
    dfx = df[df['figi'] == figi]   
 
    if dfx.shape[0] > 0 :
        res = dfx['name'].iloc[0]      
        return res
    else :
        return None
    
    
def money_value(price):
    return price.units + price.nano / 1e9

#### База идентификаторов тиньков

In [59]:
base = get_id_base(token)

In [60]:
res = []
for pos in port.positions:
    ticker = figi_to_ticker(pos.figi, base)
    name = figi_to_name(pos.figi, base)
#    print("Figi: ", pos.figi, ticker, name, "Количество: ", pos.quantity.units, "Цена: ", pos.current_price.units + pos.current_price.nano / 1e9)
    
    res.append({
        'figi': pos.figi,
        'ticker': ticker,
        'name': name,
        'quantity' : pos.quantity.units,
        'price' : money_value(pos.current_price)
    })

df_port = pd.DataFrame(res)    
df_port = df_port.sort_values("ticker")    
df_port

,figi,ticker,name,quantity,price
3,BBG007N0Z367,AGRO,РусАгро,16,1391.000
8,BBG004S686N0,BANEP,Башнефть - привилегированные акции,29,2116.500
12,BBG00475K6C3,CHMF,Северсталь,39,1774.200
0,BBG004731032,LKOH,ЛУКОЙЛ,6,7303.000
5,BBG004S68507,MAGN,Магнитогорский металлургический комбинат,740,54.650
1,BBG004S681B4,NLMK,НЛМК,390,214.140
13,BBG004731354,ROSN,Роснефть,37,554.200
2,TCS00A107RZ0,RU000A107RZ0,ГК Самолет выпуск 13,40,996.000
11,BBG004730N88,SBER,Сбер Банк,220,292.990
7,BBG004S684M6,SIBN,Газпром нефть,27,737.950


### Figi to tiker

#### Получение исторических данных

In [61]:
def get_candles(figi, interval):
    res = []
    with Client(token) as client:
        settings = MarketDataCacheSettings(base_cache_dir=Path("D:\Data\Tink\market_data_cache"))
        market_data_cache = MarketDataCache(settings=settings, services=client)
        for candle in market_data_cache.get_all_candles(
            figi = figi,
            from_=now() - timedelta(days=180),
            interval= interval,
        ):
            price_row = [candle.open,  candle.high, candle.low, candle.close]
            price_row = [money_value(x) for x in price_row]
            res.append([candle.time] + price_row)    
        return res       
    
def get_open_price(candles):
    res = []
    for row in candles:
        sdate = row[0]
        sdate = sdate.strftime("%Y-%m-%d")
        res.append([sdate] + row[1:2]) 
    df = pd.DataFrame(res, columns = ['date', 'ticker'])
    df = df.set_index('date')
    
    return df

### Load account data

In [62]:
res = []

for pos in port.positions:
    print(pos.figi)        
    candles = get_candles(pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  get_open_price(candles)
    ticker = figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)

File D:\Data\Tink\market_data_cache\BBG004731032\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG004S681B4\CANDLE_INTERVAL_DAY.meta was not found. Creating default.


BBG004731032
BBG004S681B4


File D:\Data\Tink\market_data_cache\TCS00A107RZ0\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG007N0Z367\CANDLE_INTERVAL_DAY.meta was not found. Creating default.


TCS00A107RZ0
BBG007N0Z367


File D:\Data\Tink\market_data_cache\BBG00475KHX6\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG004S68507\CANDLE_INTERVAL_DAY.meta was not found. Creating default.


...

BBG004S686N0
BBG004S681M2


File D:\Data\Tink\market_data_cache\RUB000UTSTOM\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG004730N88\CANDLE_INTERVAL_DAY.meta was not found. Creating default.


RUB000UTSTOM
BBG004730N88


File D:\Data\Tink\market_data_cache\BBG00475K6C3\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG004731354\CANDLE_INTERVAL_DAY.meta was not found. Creating default.


BBG00475K6C3
BBG004731354


In [64]:
dfp = dfp.dropna(axis = 1)
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,SBER,CHMF,ROSN
date,,,,,,,,,,,
2023-09-27,6412.0,209.46,1082.8,52.025,605.7,669.80,1402.0,49.850,256.20,1374.8,520.30
2023-09-28,6502.0,207.36,1138.8,52.100,616.5,670.50,1435.0,50.700,256.40,1380.0,531.40
2023-09-29,6650.0,210.00,1214.8,52.260,620.0,689.40,1449.5,53.080,258.00,1378.4,536.00
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,258.99,1392.0,534.65
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-18,7390.5,210.00,1374.2,55.500,723.6,812.10,2118.0,60.995,299.40,1724.4,584.00
2024-03-19,7349.0,209.16,1375.8,55.360,714.9,813.45,2105.0,61.800,298.30,1729.0,582.85
2024-03-20,7338.5,205.46,1391.4,54.920,704.5,780.00,2133.0,60.920,295.30,1720.0,565.35


### Portfolio

In [65]:

def weights_to_df(cleaned_weights):
    dfw = pd.DataFrame.from_dict([cleaned_weights]).transpose()
    dfw.columns = ['W']
    dfw = dfw[dfw['W'] > 0]
    return dfw

def final_sums(df, total):
    xx = round(df * total, -1)
    xx = xx[xx['W'] > 0]
    xx = xx.sort_values("W")
    return xx

In [66]:
avg_returns = expected_returns.mean_historical_return(dfp)
cov_mat = risk_models.exp_cov(dfp)
cov_mat

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,SBER,CHMF,ROSN
LKOH,0.039919,0.017251,0.033269,0.016690,0.016956,0.014795,0.024393,0.021929,0.018965,0.015660,0.020221
NLMK,0.017251,0.064844,0.018849,0.031013,0.010415,0.007782,0.025225,0.024968,0.017869,0.044801,0.021063
AGRO,0.033269,0.018849,0.154302,0.014490,0.015302,0.020957,0.033922,0.045902,0.028814,0.023435,0.027712
MAGN,0.016690,0.031013,0.014490,0.045256,0.009938,0.008301,0.029652,0.014577,0.015232,0.034390,0.017632
TATN,0.016956,0.010415,0.015302,0.009938,0.042442,0.013648,0.014795,0.016150,0.012147,0.010994,0.022646
SIBN,0.014795,0.007782,0.020957,0.008301,0.013648,0.057552,0.013654,0.020603,0.010947,0.004579,0.021106
BANEP,0.024393,0.025225,0.033922,0.029652,0.014795,0.013654,0.098504,0.023255,0.018423,0.027074,0.029945
SNGSP,0.021929,0.024968,0.045902,0.014577,0.016150,0.020603,0.023255,0.070490,0.021392,0.015796,0.023158
SBER,0.018965,0.017869,0.028814,0.015232,0.012147,0.010947,0.018423,0.021392,0.027548,0.017155,0.016761
CHMF,0.015660,0.044801,0.023435,0.034390,0.010994,0.004579,0.027074,0.015796,0.017155,0.057686,0.018785


In [67]:
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 10000)

Expected annual return: 103.1%
Annual volatility: 21.7%
Sharpe Ratio: 4.67


,W
SIBN,30.0
AGRO,230.0
TATN,690.0
SNGSP,1090.0
CHMF,2730.0
BANEP,5230.0


In [64]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 10000)

Expected annual return: 6.2%
Annual volatility: 4.3%
Sharpe Ratio: 0.97


,W
CNYRUB_TOM,970.0
GMKN,160.0
GAZP,20.0
PHOR,270.0
TBRU,8440.0
MTSS,100.0
ROLO,40.0


### Model

In [56]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [14]:

def calc_frontier(df_period, risk_method, ret_method = "mean_historical_return", span = 180):

    if ret_method == "ema_historical_return":
        mu = expected_returns.return_model(df_period, method=ret_method, span = span)
    else:
        mu = expected_returns.return_model(df_period, method=ret_method)
  
    cov_mat = risk_models.risk_matrix(df_period, method=risk_method)        
    ef = EfficientFrontier(mu, cov_mat)
    return ef


def calc_weights(ef, opt_type, par, verbose = False):

    if opt_type == 'max_sharpe':
        try:
            weights = ef.max_sharpe()
        except:
            if verbose:
                print("Non-convex optimize!")
            weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
        )  
    elif opt_type == 'efficient_risk':
        weights = ef.efficient_risk(par) 

    ef.portfolio_performance(verbose=verbose)
    cleaned_weights = ef.clean_weights(cutoff=0.0001)
    dfw = weights_to_df(cleaned_weights)
    dfw['W'] = dfw['W']/dfw['W'].sum()
    return dfw


In [16]:
drops = []

columns = [x for x in dfp.columns if x not in drops]
dfpx = dfp[columns]

In [15]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
ef = calc_frontier(dfpx, "ledoit_wolf",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",  "mean_historical_return")
#ef = calc_frontier(dfp, "exp_cov",  "capm_return")
 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "mean_historical_return")
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0, verbose = True)      
#dfw = calc_weights(ef, 'efficient_risk', 0.3, verbose = True) 

NameError: name 'dfpx' is not defined

In [198]:
dfx = final_sums(dfw, 10000)
dfx = dfx.sort_values("W", ascending = False)
dfx

,W
CHMF,3400.0
MTLR,2220.0
FLOT,2020.0
TBRU,1310.0
SIBN,580.0
TATN,460.0


## All shares

In [68]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]

In [69]:
res = []
for row in dfx.iterrows():
    pos = row[1]
#    print(pos.figi)        
    candles = get_candles(pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  get_open_price(candles)
    ticker = figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)


File D:\Data\Tink\market_data_cache\BBG000BN56Q9\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG000RMWQD4\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG004PYF2N3\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG000GQSVC2\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\TCS007940839\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\TCS00A0HG602\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\TCS009102396\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG000FWGSZ5\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\BBG000PKWCQ7\CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File D:\Data\Tink\market_data_cache\B

In [70]:
dfp = dfp.dropna(axis = 1)
dfp

,DSKY,ENPG,POLY,NKNCP,OBNEP,IRKT,MRKV,VSMO,UNAC,LSRG,...,CNTLP,PIKK,ROSN,MTLRP,TRMK,SOFL,MRKU,MRKY,KRKNP,CHMF
date,,,,,,,,,,,,,,,,,,,,,
2023-09-27,67.80,504.6,528.4,83.70,957.97,124.25,0.06865,45260.0,2.2000,664.4,...,14.00,752.1,520.30,223.20,225.00,145.10,0.4508,0.09550,13820.0,1374.8
2023-09-28,68.48,514.6,520.0,84.00,979.97,123.85,0.07070,45220.0,2.1825,692.0,...,14.22,755.8,531.40,225.45,227.50,150.40,0.4606,0.10395,13920.0,1380.0
2023-09-29,68.20,532.0,521.0,84.16,590.00,122.00,0.07000,45500.0,2.1120,692.0,...,14.70,754.5,536.00,226.85,220.72,200.58,0.4618,0.10200,14040.0,1378.4
2023-10-02,69.52,525.6,516.4,84.32,901.01,118.00,0.06605,43460.0,2.0400,705.4,...,14.44,742.6,538.50,226.90,223.50,218.04,0.4540,0.09900,14000.0,1371.0
2023-10-03,69.50,527.0,505.6,84.00,800.01,112.90,0.06680,42500.0,1.9900,693.6,...,14.10,736.5,534.65,235.00,221.50,207.92,0.4426,0.09535,13960.0,1392.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-18,50.94,442.2,349.4,76.64,945.97,70.50,0.06095,37320.0,1.3695,925.0,...,11.82,900.8,584.00,333.00,219.60,154.40,0.4002,0.09440,12320.0,1724.4
2024-03-19,49.82,442.0,338.4,77.16,927.01,70.00,0.06345,37060.0,1.3510,934.0,...,11.62,889.6,582.85,328.00,220.04,157.52,0.4102,0.09625,12440.0,1729.0
2024-03-20,48.64,436.0,333.2,76.06,999.99,68.30,0.06180,36780.0,1.3495,941.2,...,11.54,892.8,565.35,332.30,218.68,157.00,0.4058,0.09525,12300.0,1720.0


In [71]:
drops = ["GTRK", "SFIN", "ORUP", "UWGN", "RKKE", "KROT", "APTK", "VEON-RX"]
drops = ["SFIN", "GTRK", "NTZL", "LSRG"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

In [72]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 180)
ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",   "ema_historical_return", span = 90)

 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "ema_historical_return", span = 120)
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0, verbose = True)      
#dfw = calc_weights(ef, 'efficient_risk', 0.25, verbose = True) 

Expected annual return: 342.3%
Annual volatility: 21.7%
Sharpe Ratio: 15.66


In [73]:
dfw.sort_values("W", ascending = False)

,W
WUSH,0.35725
YNDX,0.21174
BANE,0.11233
POSI,0.10388
CHMF,0.07567
TGKBP,0.04883
AFKS,0.01459
FIVE,0.01447
OZON,0.01341
OBNEP,0.01325


In [78]:
dfx = final_sums(dfw, 113200)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

inds
x = base[base['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s


In [79]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["buy"] = np.round(dfx.W / (dfx.price * dfx.lot))
dfx["sum"]= dfx.price * dfx.buy * dfx.lot
dfx = dfx.sort_values("W", ascending = False)
dfx.to_csv("t.csv")
dfx

,W,lot,price,buy,sum
WUSH,40440.0,1,329.00000,123.0,40467.000
YNDX,23970.0,1,3885.00000,6.0,23310.000
BANE,12720.0,1,3076.50000,4.0,12306.000
POSI,11760.0,1,2680.00000,4.0,10720.000
CHMF,8570.0,1,1796.60000,5.0,8983.000
TGKBP,5530.0,100000,0.01728,3.0,5184.000
AFKS,1650.0,100,20.35000,1.0,2035.000
FIVE,1640.0,1,2879.00000,1.0,2879.000
OZON,1520.0,1,3541.00000,0.0,0.000
OBNEP,1500.0,1,958.93000,2.0,1917.860


In [76]:
dfx.sum()

W        113220.00000
lot      100121.00000
price     19379.11928
buy        9272.00000
sum      112142.29100
dtype: float64